## Libraries

In [1]:
import torch
from torch import tensor

## Broadcasting

### Example broadcast

In [2]:
c = tensor([10, 20, 30])
m = tensor([[1., 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
c.shape, m.shape

(torch.Size([3]), torch.Size([3, 3]))

In [4]:
c + m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

### Simple exercise

Suppose this is a batch of RGB images

In [5]:
image_batch = torch.rand(64, 3, 256, 256)
image_batch.shape

torch.Size([64, 3, 256, 256])

Normalize it with vectors of 3 elements. One for the mean and one for std

In [6]:
mean = torch.mean(image_batch, dim=(0, 2, 3))
mean, mean.shape

(tensor([0.4998, 0.5001, 0.5000]), torch.Size([3]))

In [7]:
std = torch.std(image_batch, dim=(0, 2, 3))
std, std.shape

(tensor([0.2887, 0.2887, 0.2887]), torch.Size([3]))

In [8]:
broadcasted_mean = mean.unsqueeze(0).unsqueeze(2).unsqueeze(3)
broadcasted_mean.shape

torch.Size([1, 3, 1, 1])

In [9]:
broadcasted_mean.storage

<bound method Tensor.storage of tensor([[[[0.4998]],

         [[0.5001]],

         [[0.5000]]]])>

In [10]:
broadcasted_std = std.unsqueeze(1).unsqueeze(1)
broadcasted_std.shape

torch.Size([3, 1, 1])

In [11]:
broadcasted_std.storage

<bound method Tensor.storage of tensor([[[0.2887]],

        [[0.2887]],

        [[0.2887]]])>

In [12]:
normalized_images = (image_batch - broadcasted_mean)/broadcasted_std
normalized_images.shape

torch.Size([64, 3, 256, 256])

## Einstein Summation

Compact representation for combining products and sums in a general way. For example:  
ik,kj->ij

Lefthand side represents the "operators" while the right side would be the result's dimensions. The rules of Einsum are:
1. Repeated indices on the left side are implicitly summed over if they are not on the right side
2. Each index can appear at most twice on the left side
3. Unrepeated indices on the left side must appear on the right side

In [13]:
x = torch.randn(2,3)
y = torch.randn(3,2)
x, y

(tensor([[-1.1307, -0.2858, -1.7678],
         [ 0.2907,  0.4672, -1.6520]]),
 tensor([[ 0.9601,  0.8377],
         [ 0.8417, -0.8874],
         [-0.3875, -1.0783]]))

### Transpose

In [14]:
torch.einsum("ij->ji", x)

tensor([[-1.1307,  0.2907],
        [-0.2858,  0.4672],
        [-1.7678, -1.6520]])

### Matrix product

In [15]:
torch.einsum('ik, kj -> ij', x, y)

tensor([[-0.6411,  1.2128],
        [ 1.3124,  1.6104]])

### Gram matrix

In [16]:
torch.einsum('ij, kj -> ik', x, x)

tensor([[4.4852, 2.4583],
        [2.4583, 3.0320]])

## Forward and backward passes

### Define and init a layer

We could stack the 2 layers on top of the other but the result of linear operations is another linear operation. To make it work we should add a nonlinearity in the middle such as the most common used activation function [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))

In [17]:
def lin(x, w, b): return x @ w + b 

We won't actually train here, so we only set random valued matrices

In [18]:
x = torch.randn(200, 100)
y = torch.randn(200)

In [19]:
w1 = torch.randn(100, 50)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1)
b2 = torch.zeros(1)

The result of our first linear layer would be:

In [20]:
l1 = lin(x, w1, b1)
l1.shape

torch.Size([200, 50])

But check how the mean and std change **AFTER** the layer. Take into account how torch.randn works:
> torch.randn: Returns a tensor filled with random numbers from a normal distribution with mean *0* and variance *1* (also called the standard normal distribution).

In [21]:
w1.mean(), w1.std(), w2.mean(), w1.std()

(tensor(0.0050), tensor(1.0085), tensor(0.0894), tensor(1.0085))

In [22]:
l1.mean(), l1.std()

(tensor(0.0242), tensor(10.0081))

Notice we are getting 10 times the original std we'd get from the original torch.randn. This is because of the existing addition in matrix multiplication. Basically, we are adding 100 values with mean 0. The mean itself will NOT change, but the spread of the values will be hugely increased after a single layer as seen. A more exaggerated version would be as follows:

In [23]:
test = torch.randn(10000,10000)
test = test @ test
test.mean(), test.std()

(tensor(0.0030), tensor(100.0119))

In [24]:
x = torch.randn(200,100)
for _ in range(50): x = x @ torch.randn(100,100)
x[0:5, 0:5], x.mean(), x.std()

(tensor([[nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan]]),
 tensor(nan),
 tensor(nan))

Since we are dealing with floats, give it a big enough number and we go to *** xddd

### Fixing initialization problems

#### [Xavier Glorot and Yoshua Bengio's weights init scaling](https://oreil.ly/9tiTC)
Basically scale to a given linear layer in order to keep std=1:
$$
\sqrt\frac{1}{n}\\
$$
With *n* being the number of inputs. In our case, with we'd have 0.1

In [25]:
x = torch.randn(200,100)
for _ in range(50): x = x @ (0.1 * torch.randn(100,100))
x[0:5, 0:5], x.mean(), x.std()

(tensor([[-0.3081, -1.0823, -1.0933, -0.5763, -0.6330],
         [-0.6660,  0.4010, -0.3800, -0.1797, -0.2145],
         [-0.4112, -0.7967, -0.7344, -0.6401, -0.5505],
         [ 0.4466,  1.1501,  0.7380,  0.5851,  0.1092],
         [ 0.3216, -0.2872, -0.2907,  0.0939, -0.1568]]),
 tensor(-0.0017),
 tensor(0.8171))

Where we can see we are keeping the proper std, mean and values. **Notice that even adding 0.01 to the scale factor the std will go up to 100! Sometimes even with the rectification it will change around 30% 1.0 value!!!**
> Now test second layer with this initialization

In [26]:
from math import sqrt
x = torch.randn(200,100)
w1 = torch.randn(100, 50) / sqrt(100)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1) / sqrt(50)
b2 = torch.zeros(1)

In [27]:
l1 = lin(x, w1, b1)
l1.mean(), l1.std()

(tensor(0.0016), tensor(1.0020))

In [28]:
def relu(x): return x.clamp_min(0.)

In [29]:
l2 = relu(l1)
l2.mean(), l2.std()

(tensor(0.3993), tensor(0.5838))

#### [Corrected init taking ReLU into account](https://oreil.ly/-_quA)
We can see that even with the correction, after ReLU we get that both mean and std have moved. In the paper linked, we can see the computation for the proper initialization taking ReLU into account:
$$
\sqrt\frac{2}{n}\\
$$

In [30]:
x = torch.randn(200,100)
for _ in range(50): x = relu(x @ (torch.randn(100,100) * sqrt(2/100)))
x[0:5, 0:5], x.mean(), x.std()

(tensor([[0.0000, 0.1243, 0.0000, 0.0000, 0.1398],
         [0.0000, 0.2184, 0.0000, 0.0000, 0.1575],
         [0.0000, 0.2943, 0.0000, 0.0000, 0.2708],
         [0.0000, 0.3140, 0.0000, 0.0000, 0.1902],
         [0.0000, 0.2908, 0.0000, 0.0000, 0.1748]]),
 tensor(0.3451),
 tensor(0.4868))

Given that's better, the book follows with creating the model. But for more normalization steps read [BatchNorm](https://arxiv.org/abs/1502.03167) or even [the following article](https://medium.com/biased-algorithms/batch-normalization-alternatives-layernorm-and-instancenorm-52bdf43624b9)

In [31]:
x = torch.randn(200,100)

In [32]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [33]:
out = model(x)
out.shape

torch.Size([200, 1])

Remember MSE (Mean Squared Error):
$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

In [34]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean() # Squeeze because of that single dim on cols

In [35]:
loss = mse(out, y)
loss

tensor(1.4403)

## Gradient and backward pass

### Regla de la cadena

#### Explicación

La regla de la cadena es una fórmula utilizada para calcular la derivada de una función compuesta. Si tenemos dos funciones, *f(x)* y *g(x)*, la derivada de su composición *f(g(x))* se puede calcular mediante la regla de la cadena. La fórmula general es:

$$
   \frac{d}{dx} \left[ f(g(x)) \right] = f'(g(x)) \cdot g'(x)
$$
Donde:
-  *f'(g(x))* es la derivada de la función externa, evaluada en *g(x)*.
- *g'(x)* es la derivada de la función interna *g(x)* respecto de *x*.

#### Ejemplo

Si ***f(x) = sin(x)* y *g(x) = x^2***, la derivada de la función compuesta ***f(g(x)) = sin(x^2)*** se obtiene aplicando la regla de la cadena:

$$
   \frac{d}{dx} \left[ \sin(x^2) \right] = \cos(x^2) \cdot 2x
$$

En este caso:
- La derivada de la función externa *f'(x) = cos(x)*, evaluada en *x^2*, es *cos(x^2)*.
- La derivada de la función interna *g'(x) = 2x*.

Entonces, la derivada de la composición es *2x · cos(x^2)*.

### Sobre su uso
La regla de la cadena se usa constantemente para calcular los gradientes en una red neuronal. Esto se debe a que, en sí mismo, ***el gradiente es el resultado de la derivada parcial con respecto a cada peso.*** La forma en la que se usa este gradiente para minimizar el error de la red neuronal mediante el descenso por gradiente:

**Descenso por gradiente:**  
Se actualiza cada peso mediante la sustracción de el learning rate multiplicado por el gradiente

$$
w_{\text{new}} = w_{\text{old}} - \eta \cdot \frac{\partial L}{\partial w}
$$

Where:
$$
w_{\text{new}} \text{is the updated weight.}\\\
w_{\text{old}} \text{is the current weight.}\\\
\eta \ \text{is the **learning rate** (a hyperparameter that controls how much we adjust the weights).}\\\
\frac{\partial L}{\partial w} \ \text{is the gradient, i.e., the partial derivative of the loss function with respect to the weight.}\\\
$$

En resumen:
- El **gradiente** dice en qué dirección debe moverse el peso.
- El **learning rate** controla el tamaño de cada paso.

Nota:
- En aplicaciones más avanzadas, se puede incorporar conceptos como momento

### En este caso particular:

#### Cita libro

The chain rule tells us that we have:
$$\frac{\text{d} loss}{\text{d} b_{2}} = \frac{\text{d} loss}{\text{d} out} \times \frac{\text{d} out}{\text{d} b_{2}} = \frac{\text{d}}{\text{d} out} mse(out, y) \times \frac{\text{d}}{\text{d} b_{2}} lin(l_{2}, w_{2}, b_{2})$$

Básicamente indicándonos que tenemos que hacer la derivada del loss con respecto a la salida de la red neuronal, es decir, la derivada de mse con respecto a las activaciones finales y luego la derivada de estas activaciones con respecto a los biases que añadimos en la segunda capa.

#### Aplicado a nuestro caso

##### La derivada del MSE

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

$$\text{Where:}\\\ - \hat{y_i} \text{: Es el objetivo real}\\\ - y \text{: Es el resultado obtenido de la red neuronal}$$

Apliquemos que:$$z = y_i - \hat{y_i}$$

La derivada utilizando la regla de la cadena dada esa aplicación sería:
$$\Delta{y_i} = \frac{dloss}{dy_i} = \frac{d\text{MSE}}{dy_i} = \frac{d\text{MSE}}{dz}·\frac{dz}{dy_i}$$
$$\frac{dloss}{dy_i} = \frac{d\text{MSE}}{dy_i} = \frac{1}{n}·2z·\frac{dz}{dy_i}$$
$$\frac{d\text{MSE}}{dy_i} = \frac{2z}{n}·\frac{dz}{dy_i}$$
$$\frac{d\text{MSE}}{dy_i} = \frac{2y_i}{n}$$
**Expresada en código:**

In [36]:
# Recordemos: def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean() # Squeeze because of that single dim on cols
# También, la salida de la capa es un tensor de dimensiones (x, 1) donde x es el tamaño de la capa
def mse_grad(layer_output, target):
    return (2 / layer_output.shape[0]) * (layer_output.squeeze() - target).unsqueeze(-1) # En realidad se usa ese unsqueeze(-1) al final para mentener la forma de columna

##### La derivada parcial para la última capa (ReLu)

**ReLU y su derivada:**
$$\text{ReLU}(x) = \max(0, x)$$
$$
\frac{dReLu(x)}{dx} = 
\begin{cases} 
1 & \text{if } x > 0 \\
0 & \text{if } x \leq 0
\end{cases}
$$

**Derivada parcial respecto al peso de la capa ReLu:**
$$\Delta{y_i} = \frac{dloss}{dy_i} = \frac{d\text{MSE}}{dy_i} = \frac{2y_i}{n}$$
$$\frac{dloss}{dw^{[ReLu]}_j} = \Delta{y_i} · \frac{dReLu(w^{[ReLu]}_j)}{dw^{[ReLu]}_j}$$
$$
\frac{dloss}{dw^{[ReLu]}_j} = \Delta{y_i} · 
\begin{cases} 
1 & \text{if } w^{[ReLu]}_j > 0 \\
0 & \text{if } w^{[ReLu]}_j \leq 0
\end{cases}
$$
**En código:**

In [37]:
def relu_grad(inp, out):
    # Cuando se hace un tensor > int se aplica el booleano al tensor completo, devolviendo un tensor de ese tamaño. Cuando se hace .float() se devuelve 1.0 o 0.0
    # Nos interesa utilizar el gradiente que ya hemos calculado para el resultado, porque en caso contrario tendríamos que usar la fórmula entera para calcular el gradiente de esta capa incluyendo el gradiente de la final
    inp.g = (inp>0).float() * out.g

##### La derivada parcial para la primera capa (Multiplicación matricial)
[Para entender de donde viene esta derivada](https://math.stackexchange.com/questions/1866757/not-understanding-derivative-of-a-matrix-matrix-product)

In [38]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

### Forward and backward passes

In [39]:
def forward_and_backward(inp, targ):
    # Forward pass
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # We don't actually need the loss in backward!
    loss = mse(out, targ)

    # Backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

## Refactoring model in Pytorch

Mejor crear clases para las funciones de activación para llamar a los forward y backward passes. Como tienen muchas cosas en común se puede hacer que hereden de una clase padre que bien podría llamarse FuncionCapa o LayerFunction
> Nota: La keyword **__call__** en Python te sirve para hacer que esa clase pueda ser llamada. Por ejemplo:
```python
class myClass():
    __init__(self):
        pass

    __call__(self, args):
        # Do somethin
        pass

x = myClass()
results = x(args)
```

#### Vamos a refactorizar de golpe a Pytorch.

En Pytorch, debes elegir lo qué guardar en una variable de contexto. Así se aseguran de que no guardamos nada innecesario. Además, **return debe devolver los gradientes en la función backward**. A continuación cómo escribiríamos esta red:

In [40]:
from torch.autograd import Function

class MyRelu(Function):
    @staticmethod
    def forward(ctx, i):
        result = i.clamp_min(0.)
        ctx.save_for_backward(i)
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        i, = ctx.saved_tensors
        return grad_output * (i>0).float()

Esta es la forma en la que escribiríamos activaciones completamente personalizadas. A continuación, se muestra la estructura que usarías para poder usar estos hijos de *Function*. Para ello, se debe crear una clase que herede de **nn.Module.** Esta es la estructura general con la cual podríamos definir cualquier modelo personalizado. Para implementarlo se debe:
1. Asegurar que superclass __init__ se llama lo primero al inicializarlo.
2. Definir cualquier parámetro del modelo como atributo con nn.Parameter.
3. Definir una función *forward* que retorne la salida del modelo.

In [41]:
import torch.nn as nn

class LinearLayer(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_out, n_in) * sqrt(2/n_in))
        self.bias = nn.Parameter(torch.zeros(n_out))
    
    def forward(self, x): return x @ self.weight.t() + self.bias

## Ejemplo modelo completo en Pytorch

```python
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)
```